# Mi primer web scraper
## Motivación
Christopher es un joven programador egresado de la Pontificia Universidad Católica del Perú que busca mejorar sus habilidades en Python. Uno de sus amigos le ha comentado que una buena forma de dominar un lenguaje es haciendo pull requests en repositorios de GitHub.

Sin embargo, revisar constantemente los nuevos repositorios le quita tiempo a Christopher, quien trabaja en un importante banco del país y además es desarrollador freelance. Por ello, Christopher le solicita a usted ayuda creando un proyecto que le permita saber cuáles son los repositorios de GitHub escritos en Python más populares.

## Descripción del proyecto
Aplicaremos los conceptos vistos en clase para desarrollar un web scraper sencillo que nos permita obtener el nombre y la descripción de los repositorios más populares de GitHub que usen Python como lenguaje.

### Importando librerías
Importamos las tres librerías cuyo uso hemos visto en clase

In [0]:
import requests
from bs4 import BeautifulSoup
import re

### Paso Uno: Recibir el Código HTML
Utilizamos la librería ```requests``` para poder extraer el código HTML de la página Trending de GitHub, ubicada en la dirección https://github.com/trending

Para esta sección debemos tener en cuenta los distintos códigos de estado en una petición HTTP: https://www.restapitutorial.com/httpstatuscodes.html

In [0]:
page = requests.get('https://github.com/trending')
page.content

b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-/YEVWs7BzxfKyUd6zVxjEQcXRWsLbcEjv045Rq8DSoipySmQblhVKxlXLva2GtNd5DhwCxHwW1RM0N9I7S2Vew==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-481a47a96965f6706fb41bae0d14b09a.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-jBbjd8Z1ZIbmMjdUmZgKUXzYfCt2DP7sPnqR5//mM51mnQ4UoeRl2HRtf

Juguemos un poco con las opciones que nos brinda la librería.

In [0]:
page.headers

{'Date': 'Wed, 13 Nov 2019 20:36:49 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '200 OK', 'Vary': 'X-PJAX, Accept-Encoding', 'ETag': 'W/"9740f669a5589c6db0613fdd3a1f555a"', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Set-Cookie': 'has_recent_activity=1; path=/; expires=Wed, 13 Nov 2019 21:36:48 -0000, _octo=GH1.1.245500941.1573677408; domain=.github.com; path=/; expires=Sat, 13 Nov 2021 20:36:48 -0000, logged_in=no; domain=.github.com; path=/; expires=Sun, 13 Nov 2039 20:36:49 -0000; secure; HttpOnly, _gh_sess=ZXNyYTZQcDlLQkZuZlJBWWQ2R0daVSttUE51OTBPR1pCSVUxYVhZclFjb1lBVWxZQktlSGpTR0dUSlRIcEVGeEFsSmZCb2dWOXVwNG9pY1lPeU12VHlRUU5qRi9UZ0RqdkRoSWFVb2F5S1I2bUJMSUNIRjIxaFR5MXdnQ2xCd3c2bExPRnZZK2pVcjdBakFPeUsyVWdBPT0tLTZxTERoYVBRM1YwR3M0bVY4RDIyb0E9PQ%3D%3D--a533615f83175e662eb01285f2d16a55f8cc6156; path=/; secure; HttpOnly', 'X-Request-Id': '5f755714-58d0-45bd-aca6-3312b4eddc03', 'Strict-Transport-Security':

In [0]:
for key in page.headers:
  print(key,'->',page.headers[key])

Date -> Wed, 13 Nov 2019 20:36:49 GMT
Content-Type -> text/html; charset=utf-8
Transfer-Encoding -> chunked
Server -> GitHub.com
Status -> 200 OK
Vary -> X-PJAX, Accept-Encoding
ETag -> W/"9740f669a5589c6db0613fdd3a1f555a"
Cache-Control -> max-age=0, private, must-revalidate
Set-Cookie -> has_recent_activity=1; path=/; expires=Wed, 13 Nov 2019 21:36:48 -0000, _octo=GH1.1.245500941.1573677408; domain=.github.com; path=/; expires=Sat, 13 Nov 2021 20:36:48 -0000, logged_in=no; domain=.github.com; path=/; expires=Sun, 13 Nov 2039 20:36:49 -0000; secure; HttpOnly, _gh_sess=ZXNyYTZQcDlLQkZuZlJBWWQ2R0daVSttUE51OTBPR1pCSVUxYVhZclFjb1lBVWxZQktlSGpTR0dUSlRIcEVGeEFsSmZCb2dWOXVwNG9pY1lPeU12VHlRUU5qRi9UZ0RqdkRoSWFVb2F5S1I2bUJMSUNIRjIxaFR5MXdnQ2xCd3c2bExPRnZZK2pVcjdBakFPeUsyVWdBPT0tLTZxTERoYVBRM1YwR3M0bVY4RDIyb0E9PQ%3D%3D--a533615f83175e662eb01285f2d16a55f8cc6156; path=/; secure; HttpOnly
X-Request-Id -> 5f755714-58d0-45bd-aca6-3312b4eddc03
Strict-Transport-Security -> max-age=31536000; includeSubdo

In [0]:
page.encoding

'utf-8'

In [0]:
page.url

'https://github.com/trending'

In [0]:
page.status_code

200

In [0]:
print(requests.codes.ok)
if page.status_code == requests.codes.ok:
  print('Todo OK!')

200
OK!


### Paso Dos: Estructurar el código HTML

Como podemos apreciar, el código HTML que recibimos es una larga cadena de texto y es imposible obtener ningún tipo de insight así. Por ello es que necesitamos darle una estructura al contenido del documento.

In [0]:
content = BeautifulSoup(page.content, 'html.parser')

content


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-481a47a96965f6706fb41bae0d14b09a.css" integrity="sha512-/YEVWs7BzxfKyUd6zVxjEQcXRWsLbcEjv045Rq8DSoipySmQblhVKxlXLva2GtNd5DhwCxHwW1RM0N9I7S2Vew==" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-1ff4ff5e4f0093fe173c3b2e49497e30.css" integrity="sha512-jBbjd8Z1ZIbmMjdUmZgKUXzYfCt2D

Exploremos un poco más las opciones que ofrece BeautifulSoup

In [0]:
content.title
#content.title.text
#content.title.name
#content.title.string
#content.title.parent
#content.p
#content.p.parent
#content.p.name
#content.p['class']

In [0]:
#content.find_all('a')
#content.find('a')
#content.find_all('a','px-2')
#content.find_all('a',{'class':'px-2','href':'#start-of-content'})
#content.find('a').attrs

{'class': ['px-2',
  'py-4',
  'bg-blue',
  'text-white',
  'show-on-focus',
  'js-skip-to-content'],
 'href': '#start-of-content',
 'tabindex': '1'}

In [0]:
for link in content.find_all('a'):
    print(link.get('href'))

In [0]:
print(content.get_text())

### Paso Tres: Encontrar (y seleccionar) las secciones que nos interesan
Teniendo un documento estructurado con BeautifulSoup, es momento de hacer un análisis de la misma estructura. Para ello, debemos apoyarnos tanto del output del Paso anterior como de la herramienta Inspeccionar Elemento de nuestro navegador.

Una vez hayamos definido qué etiquetas, categorías y/o atributos definen la información que queremos obtener, procedemos a generar nuestro selector

In [0]:
titles = content.select('.Box-row h1 [href]')
descriptions = content.select('.Box-row p')
languages = content.select('[itemprop=programmingLanguage]')

Antes de continuar, debemos validar que estemos extrayendo la información adecuada y no se nos escape ninguna celda. Para ello podemos imprimir la cantidad de elementos en cada una de nuestras listas y compararla a los elementos en la página Trending.

Una vez confirmamos que no hay diferencias (o que las diferencias están presentes de manera natural, como en el caso de Repositorios sin un lenguaje de programación especificado) podemos proceder al siguiente paso.

In [0]:
print(len(titles))
print(len(descriptions))
print(len(languages))

25
25
23


Sin embargo, no podemos trabajar con esta situación puesto que no hay forma de saber cuáles son los elementos sin descripción o sin lenguaje de programación definido. Para poder saltarnos esta situación tenemos que redefinir nuestro algoritmo.

In [0]:
boxrows = content.select('.Box-row')

titles = []
descriptions = []
languages = []

for elem in boxrows:
  # Titulos
  temp = elem.select('h1 [href]')
  if len(temp) > 0:
    titles.append(temp[0])
  else:
    titles.append(None)
  # Descripciones
  temp = elem.select('p')
  if len(temp) > 0:
    descriptions.append(temp[0])
  else:
    descriptions.append(None)
  # Lenguajes
  temp = elem.select('[itemprop=programmingLanguage]')
  if len(temp) > 0:
    languages.append(temp[0])
  else:
    languages.append(None)

### Paso Cuatro: Extraer específicamente el texto que nos interesa utilizando RegEx
Como pueden comprobar en nuestras colecciones de títulos, descripciones y lenguajes, hay mucho trabajo de limpieza por hacer. Para este trabajo utilizaremos la funcionalidad que ofrece RegEx con su librería re. En un caso particular, aprovecharemos la funcionalidad de BeautifulSoup4 al trabajar con Tags.

In [0]:
new_titles = []
new_descriptions = []
new_languages = []

for elem in titles:
  temp = re.findall('<a href="(.*)">',str(elem))[0]
  new_titles.append(temp)

for elem in descriptions:
  if elem is None:
    new_descriptions.append(None)
  else:
    temp = re.findall('\n *(.*)\n *</p>',str(elem))[0]
    new_descriptions.append(temp)
    
for elem in languages:
  if elem is None:
    new_languages.append(None)
  else:
    temp = elem.contents[0]
    new_languages.append(temp)

Terminamos nuestro trabajo con tres listas ordenadas: una con nombres de repositorio, otra con las descripciones respectivas y otra con el lenguaje del repositorio. Podríamos dar por terminada esta tarea, pero queremos darle a Christopher un buen trabajo, por lo que utilizaremos la librería Pandas para poder mostrarle un reporte final.

### Bonus: Armar un DataFrame con Pandas y filtrar la información
La librería Pandas es la más conocida para mostrar y trabajar sobre data estructurada y será la que utilicemos para mostrar los resultados de nuestro scraping. En este caso, mostraremos una tabla con todos los proyectos en la categoría Trending y además filtraremos solo los proyectos en lenguaje Python.

In [0]:
import pandas as pd

df = pd.DataFrame([new_titles,new_descriptions,new_languages]).transpose()
df.columns = ['Repositorio','Descripcion','Lenguaje']
df

,Repositorio,Descripcion,Lenguaje
0,/quantopian/zipline,"Zipline, a Pythonic Algorithmic Trading Library",Python
1,/2227324689/gpmall,【咕泡学院实战项目】-基于SpringBoot+Dubbo构建的电商平台-微服务架构、商城、...,Java
2,/sinclairzx81/zero,3D graphics rendering pipeline. Implemented in...,TypeScript
3,/ripienaar/free-for-dev,"A list of SaaS, PaaS and IaaS offerings that h...",HTML
4,/bregman-arie/devops-interview-questions,"Linux, Jenkins, AWS, Network, Prometheus, Dock...",Groovy
5,/mitesh77/Best-Flutter-UI-Templates,completely free for everyone. Its build-in Flu...,Dart
6,/trimstray/nginx-admins-handbook,"How to improve NGINX performance, security, an...",Shell
7,/donnemartin/data-science-ipython-notebooks,Data science Python notebooks: Deep learning (...,Python
8,/yifeikong/reverse-interview-zh,技术面试最后反问面试官的话,None
9,/qazbnm456/awesome-web-security,"<g-emoji alias=""dog"" class=""g-emoji"" fallback-...",None


Hacemos el filtro que nos solicita Christopher para terminar con esta actividad

In [0]:
df[df['Lenguaje'] == 'Python']

,Repositorio,Descripcion,Lenguaje
0,/quantopian/zipline,"Zipline, a Pythonic Algorithmic Trading Library",Python
7,/donnemartin/data-science-ipython-notebooks,Data science Python notebooks: Deep learning (...,Python
11,/nvbn/thefuck,Magnificent app which corrects your previous c...,Python
12,/vinta/awesome-python,"A curated list of awesome Python frameworks, l...",Python
13,/evilsocket/pwnagotchi,(⌐■_■) - Deep Reinforcement Learning instrumen...,Python
14,/notadamking/tensortrade,An open source reinforcement learning framewor...,Python
